In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('ch5').getOrCreate()

In [0]:
from pyspark.sql.types import LongType
def cubed(a):
  return a**3
spark.udf.register("cubing",cubed,LongType())
spark.range(1,9).createOrReplaceTempView('udf_test')

In [0]:
spark.sql("""select id, cubing(id) from udf_test""").show()

+---+----------+
 id|cubing(id)|
+---+----------+
 1| 1|
 2| 8|
 3| 27|
 4| 64|
 5| 125|
 6| 216|
 7| 343|
 8| 512|
+---+----------+

In [0]:
from pyspark.sql.types import *
schema = StructType([StructField('celsius',ArrayType(IntegerType()))])
t_list=[[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
t_c=spark.createDataFrame(t_list,schema)
t_c.createOrReplaceTempView('tC')
t_c.show()

+--------------------+
 celsius|
+--------------------+
[35, 36, 32, 30, ...|
[31, 32, 34, 55, 56]|
+--------------------+

In [0]:
#celsius to fahrenheit
spark.sql('''select celsius, transform(celsius,t->((t*9) div 5)+32) as fahrenheit from tC''').show()
#NOTE: im yet to see the alternate approach for above, i.e im not able to create the same using df approach.

+--------------------+--------------------+
 celsius| fahrenheit|
+--------------------+--------------------+
[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+

In [0]:
#NOTE: below code does not work
#t_c2= t_c.transform(t_c['celsius'],t=>((t*9)/5)+32)
#t_c2.toDF.show()

In [0]:
#Filter temperatures > 38C for array of temperatures
spark.sql("""select celsius, filter(celsius, t->t>38) as highTemp from tC""").show()

+--------------------+--------+
 celsius|highTemp|
+--------------------+--------+
[35, 36, 32, 30, ...|[40, 42]|
[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+

In [0]:
#Is there a temperature of 38C in the array of temperatures
spark.sql("""select celsius, exists(celsius,t->t=38) as has38 from tC""").show()

+--------------------+-----+
 celsius|has38|
+--------------------+-----+
[35, 36, 32, 30, ...| true|
[31, 32, 34, 55, 56]|false|
+--------------------+-----+

In [0]:
#Common DataFrames and SQL Operations
from pyspark.sql.functions import expr 
tripdelaysFilePath ="/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
airportsnaFilePath ="/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt"

In [0]:
airportna = spark.read.format('csv').options(header='true',inferSchema='true',sep='\t').load(airportsnaFilePath)
airportna.show(5)

+----------+-----+-------+----+
 City|State|Country|IATA|
+----------+-----+-------+----+
Abbotsford| BC| Canada| YXX|
 Aberdeen| SD| USA| ABR|
 Abilene| TX| USA| ABI|
 Akron| OH| USA| CAK|
 Alamosa| CO| USA| ALS|
+----------+-----+-------+----+
only showing top 5 rows

In [0]:
departureDelays=spark.read.csv(tripdelaysFilePath,header=True)
departureDelays.show(5)

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01011245| 6| 602| ABE| ATL|
01020600| -8| 369| ABE| DTW|
01021245| -2| 602| ABE| ATL|
01020605| -4| 602| ABE| ATL|
01031245| -4| 602| ABE| ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows

In [0]:
departureDelays.printSchema()

root
-- date: string (nullable = true)
-- delay: string (nullable = true)
-- distance: string (nullable = true)
-- origin: string (nullable = true)
-- destination: string (nullable = true)

In [0]:
departureDelays= departureDelays.withColumn('delay',expr("CAST(delay as INT) as delay")).withColumn('distance',expr("CAST(distance as INT) as distance"))
departureDelays.printSchema()

root
-- date: string (nullable = true)
-- delay: integer (nullable = true)
-- distance: integer (nullable = true)
-- origin: string (nullable = true)
-- destination: string (nullable = true)

In [0]:
departureDelays.createOrReplaceTempView('departureDelays')
airportna.createOrReplaceTempView('airports_na')

In [0]:
foo=departureDelays.filter(expr("""origin == 'SEA' and destination == 'SFO' and date like '01010%' and delay > 0"""))
foo.show(5)

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
+--------+-----+--------+------+-----------+

In [0]:
foo.createOrReplaceTempView('foo')

In [0]:
spark.sql("select * from airports_na limit 10").show()

+-----------+-----+-------+----+
 City|State|Country|IATA|
+-----------+-----+-------+----+
 Abbotsford| BC| Canada| YXX|
 Aberdeen| SD| USA| ABR|
 Abilene| TX| USA| ABI|
 Akron| OH| USA| CAK|
 Alamosa| CO| USA| ALS|
 Albany| GA| USA| ABY|
 Albany| NY| USA| ALB|
Albuquerque| NM| USA| ABQ|
 Alexandria| LA| USA| AEX|
 Allentown| PA| USA| ABE|
+-----------+-----+-------+----+

In [0]:
spark.sql("select * from foo").show()

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
+--------+-----+--------+------+-----------+

In [0]:
#Union example
bar = departureDelays.union(foo)

In [0]:
bar.filter(expr("""origin=='SEA' and destination == 'SFO' and date like '01010%' and delay > 0""")).show()

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
+--------+-----+--------+------+-----------+

In [0]:
bar.createOrReplaceTempView('bar')
spark.sql("select * from bar where origin == 'SEA' and destination == 'SFO' and date like '01010%' and delay>0").show()

+--------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
01010710| 31| 590| SEA| SFO|
01010955| 104| 590| SEA| SFO|
01010730| 5| 590| SEA| SFO|
+--------+-----+--------+------+-----------+

In [0]:
foo.join(airportna,foo.origin==airportna.IATA).select('City','State','delay','distance','destination').show()

+-------+-----+-----+--------+-----------+
 City|State|delay|distance|destination|
+-------+-----+-----+--------+-----------+
Seattle| WA| 31| 590| SFO|
Seattle| WA| 104| 590| SFO|
Seattle| WA| 5| 590| SFO|
+-------+-----+-----+--------+-----------+

In [0]:
spark.sql('select a.City,a.State,b.delay,b.distance,b.destination from airports_na as a join foo as b on a.IATA=b.origin').show()

+-------+-----+-----+--------+-----------+
 City|State|delay|distance|destination|
+-------+-----+-----+--------+-----------+
Seattle| WA| 31| 590| SFO|
Seattle| WA| 104| 590| SFO|
Seattle| WA| 5| 590| SFO|
+-------+-----+-----+--------+-----------+

In [0]:
#Adding New Column
foo2 = foo.withColumn('status',expr("""case when delay <= 10 then 'On-Time' ELSE 'Delayed' END"""))
foo2.show()

+--------+-----+--------+------+-----------+-------+
 date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
01010710| 31| 590| SEA| SFO|Delayed|
01010955| 104| 590| SEA| SFO|Delayed|
01010730| 5| 590| SEA| SFO|On-Time|
+--------+-----+--------+------+-----------+-------+

In [0]:
foo3=foo2.drop('delay')
foo3.show()

+--------+--------+------+-----------+-------+
 date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
01010710| 590| SEA| SFO|Delayed|
01010955| 590| SEA| SFO|Delayed|
01010730| 590| SEA| SFO|On-Time|
+--------+--------+------+-----------+-------+